<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=975c7d2cf7b538d5bf3c561c6dc92f39ae3ea5cae47c378f32eeedf48ac89d23
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf


    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-26 09:43:00
-------------------
qualified stocks: 87
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  4.38 K
Current:  1.37 C
-------------------
Today PnL: -1.40 L (-1.01%)
Current PnL: -22.42 L (-15.08%)
CY Booked + Current PnL: -10.98 L (-7.39%)
-------------------
Total profit:  96.95 K
Total loss:  -23.39 L
-------------------
Total Booked + Current PnL: 16.00 L (13.15%)
Total Booked PnL: 38.42 L (31.57%)
Curr Year Booked PnL: 11.44 L (8.34%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.25 C
Est FTT PnL: 88.29 L (64.36%)
Deployed:  1.22 C
Current:  1.37 C
CAGR/XIRR %: 7.26%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
14,BLUESTARCO,2080.00,11.32,52.0,H-MC,3.87,192050.0,27380.0,15959.0,-0.35,16.63,8.31,26.31,89.0,1.72,1.40,25.94,X40N,NTT,AC
77,TTKPRESTIG,770.00,94.02,37.0,M-SC,2.76,83637.0,-17140.0,17229.0,-2.26,-17.01,20.60,0.09,245.0,-0.99,0.61,9.88,OX40N,NTT,DURABLES
50,NATIONALUM,244.55,-45.32,52.0,H-MC,10.98,108364.0,7860.0,21250.0,-0.56,7.82,19.61,28.96,79.0,0.37,0.79,44.44,MH,ATH,METALS
42,ITC,452.00,-39.19,38.0,H-LC,2.20,195771.0,-4367.0,25254.0,0.06,-2.18,12.90,10.44,4.0,-0.17,1.43,3.26,X40,NTT,FMCG
57,RELIANCE,1533.00,-14.79,40.0,H-LC,4.06,213424.0,3058.0,25718.0,-0.31,1.45,12.05,13.68,37.0,0.12,1.56,18.22,XY25,NTT,REFINERIES


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TATAMOTORS,1065.00,-22.82,42.0,M-LC,2.34,244408.0,-32115.0,142197.0,1.35,-11.61,58.18,39.81,54.0,-0.23,1.78,17.13,XY24,NTT,AUTO
34,ICICIGI,2260.25,-18.25,53.0,H-MC,1.87,140541.0,4408.0,26717.0,0.92,3.24,19.01,22.86,91.0,0.16,1.02,16.26,X40,ATH,INSURANCE
66,SIS,528.00,2100.52,44.0,H-SC,4.25,88804.0,-22228.0,45841.0,0.62,-20.02,51.62,21.26,156.0,-0.48,0.65,19.98,OX40N,NTT,MISC
8,AWL,485.00,-64.78,53.0,H-MC,2.99,250974.0,-50777.0,212199.0,0.44,-16.83,84.55,53.50,116.0,-0.24,1.83,11.47,XY24,NTT,FMCG
43,JCHAC,2282.00,19813.04,48.0,M-SC,1.78,97272.0,-30533.0,30524.0,0.22,-23.89,31.38,-0.01,232.0,-1.00,0.71,10.15,OX40N,NTT,AC


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WHIRLPOOL,2270.00,-44.58,32.0,M-SC,1.46,93675.0,2176.0,76579.0,-3.63,2.38,81.75,86.07,223.0,0.03,0.68,36.03,XR,NTT,DURABLES
21,COFFEEDAY,80.00,-47.69,42.0,L-SC,41.33,78720.0,-34829.0,74878.0,-3.23,-30.67,95.12,35.27,268.0,-0.47,0.57,92.67,XR,NTT,HOTELS
46,KPIGREEN,731.05,-3.26,32.0,H-SC,8.77,115441.0,-9856.0,68780.0,-2.65,-7.87,59.58,47.03,141.0,-0.14,0.84,44.54,MH,ATH,POWER
27,HAPPSTMNDS,1488.71,-20.13,28.0,H-SC,10.51,86160.0,-42005.0,152029.0,-2.63,-32.77,176.45,85.85,132.0,-0.28,0.63,0.00,AR,ATH,IT
59,ROUTE,2236.84,-50.28,30.0,H-SC,16.95,81810.0,-51256.0,144108.0,-2.34,-38.52,176.15,69.78,118.0,-0.36,0.60,0.00,SR,ATH,IT


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
37,INDIAMART,4850.92,-55.57,24.0,H-SC,10.22,124152.0,816.0,132942.0,-1.31,0.66,107.08,108.45,119.0,0.01,0.91,23.16,AR,ATH,MISC


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UJJIVANSFB,60.0,89.03,46.0,H-SC,10.12,119718.0,-22761.0,42284.0,-1.40,-15.97,35.32,13.70,163.0,-0.54,0.87,41.30,OX40N,NTT,BANKS
77,TTKPRESTIG,770.0,94.02,37.0,M-SC,2.76,83637.0,-17140.0,17229.0,-2.26,-17.01,20.60,0.09,245.0,-0.99,0.61,9.88,OX40N,NTT,DURABLES
45,KANSAINER,340.0,-68.16,48.0,H-SC,1.40,219420.0,-50247.0,86583.0,-0.59,-18.63,39.46,13.47,138.0,-0.58,1.60,10.98,XY24,NTT,PAINTS
66,SIS,528.0,2100.52,44.0,H-SC,4.25,88804.0,-22228.0,45841.0,0.62,-20.02,51.62,21.26,156.0,-0.48,0.65,19.98,OX40N,NTT,MISC
18,CERA,9475.0,-23.87,30.0,H-SC,1.89,138851.0,-37052.0,79076.0,-1.48,-21.06,56.95,23.89,149.0,-0.47,1.01,19.92,OX40N,NTT,CERAMICS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAIL,228.00,47.55,63.0,M-MC,10.34,232556.0,7594.0,160068.0,-1.08,3.38,68.83,74.53,192.0,0.05,1.70,36.72,XY24,BTT,STEEL
14,BLUESTARCO,2080.00,11.32,52.0,H-MC,3.87,192050.0,27380.0,15959.0,-0.35,16.63,8.31,26.31,89.0,1.72,1.40,25.94,X40N,NTT,AC
84,VOLTAS,1530.00,-3.04,42.0,H-MC,3.05,203385.0,11643.0,26115.0,-1.20,6.07,12.84,19.69,99.0,0.45,1.48,13.28,XY25,NTT,AC
17,CAMS,5211.76,-5.83,44.0,H-SC,1.76,107125.0,5121.0,38801.0,-1.10,5.02,36.22,43.06,122.0,0.13,0.78,23.00,X40N,ATH,MISC
79,UNIONBANK,163.00,-14.73,54.0,M-LC,6.84,155144.0,14304.0,30191.0,-1.67,10.16,19.46,31.59,66.0,0.47,1.13,39.33,XY24,NTT,BANKS


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-45.32,52.0,H-MC,10.98,108364.0,7860.0,21250.0,-0.56,7.82,19.61,28.96,79.0,0.37,0.79,44.44,MH,ATH,METALS
85,WHIRLPOOL,2270.00,-44.58,32.0,M-SC,1.46,93675.0,2176.0,76579.0,-3.63,2.38,81.75,86.07,223.0,0.03,0.68,36.03,XR,NTT,DURABLES
11,BANKINDIA,190.00,-29.19,54.0,H-MC,8.22,181776.0,1968.0,110447.0,-1.31,1.09,60.76,62.52,88.0,0.02,1.33,35.55,XR,NTT,BANKS
37,INDIAMART,4850.92,-55.57,24.0,H-SC,10.22,124152.0,816.0,132942.0,-1.31,0.66,107.08,108.45,119.0,0.01,0.91,23.16,AR,ATH,MISC
86,WIPRO,420.00,-16.06,34.0,M-LC,5.63,147796.0,-3149.0,112606.0,-1.50,-2.09,76.19,72.51,53.0,-0.03,1.08,3.81,XR,NTT,IT


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
44,JIOFIN,387.00,-5.61,29.0,H-LC,12.96,256452.0,-10913.0,76371.0,-1.13,-4.08,29.78,24.48,48.0,-0.14,1.87,48.65,XY24,BTT,FINANCE
5,ASIANPAINT,3465.66,-17.51,30.0,H-LC,7.05,208146.0,-43622.0,96830.0,-1.61,-17.33,46.52,21.13,27.0,-0.45,1.52,11.96,X40,ATH,PAINTS
74,TCS,4476.75,-30.93,30.0,H-LC,12.07,274997.0,-70959.0,145831.0,-1.08,-20.51,53.03,21.64,1.0,-0.49,2.01,0.00,X40,ATH,IT
83,VBL,671.64,-20.19,32.0,H-LC,8.17,287387.0,-28455.0,141107.0,-0.31,-9.01,49.10,35.67,5.0,-0.20,2.10,3.60,X40N,ATH,FMCG
19,CIPLA,1795.00,-23.89,34.0,H-LC,4.51,203418.0,-1082.0,42494.0,-1.65,-0.53,20.89,20.25,10.0,-0.03,1.48,8.16,X40N,BTT,PHARMA


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
48,LTIM,7230.2,-7.23,40.0,H-LC,1.68,229725.0,-16426.0,95635.0,-2.09,-6.67,41.63,32.18,16.0,-0.17,1.68,26.96,X200,ATH,IT
42,ITC,452.0,-39.19,38.0,H-LC,2.20,195771.0,-4367.0,25254.0,0.06,-2.18,12.90,10.44,4.0,-0.17,1.43,3.26,X40,NTT,FMCG
65,SIEMENS,4671.5,-2.83,43.0,H-LC,2.61,156250.0,-29845.0,77328.0,-0.99,-16.04,49.49,25.51,15.0,-0.39,1.14,15.14,AR,ATH,ELECTRICAL
57,RELIANCE,1533.0,-14.79,40.0,H-LC,4.06,213424.0,3058.0,25718.0,-0.31,1.45,12.05,13.68,37.0,0.12,1.56,18.22,XY25,NTT,REFINERIES
19,CIPLA,1795.0,-23.89,34.0,H-LC,4.51,203418.0,-1082.0,42494.0,-1.65,-0.53,20.89,20.25,10.0,-0.03,1.48,8.16,X40N,BTT,PHARMA


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4476.75,-30.93,30.0,H-LC,12.07,274997.0,-70959.0,145831.0,-1.08,-20.51,53.03,21.64,1.0,-0.49,2.01,0.00,X40,ATH,IT
80,UNITDSPR,1644.00,-14.09,45.0,H-LC,7.15,225251.0,-8893.0,57507.0,-0.61,-3.80,25.53,20.77,86.0,-0.15,1.64,2.86,X40N,NTT,BREWERIES
42,ITC,452.00,-39.19,38.0,H-LC,2.20,195771.0,-4367.0,25254.0,0.06,-2.18,12.90,10.44,4.0,-0.17,1.43,3.26,X40,NTT,FMCG
83,VBL,671.64,-20.19,32.0,H-LC,8.17,287387.0,-28455.0,141107.0,-0.31,-9.01,49.10,35.67,5.0,-0.20,2.10,3.60,X40N,ATH,FMCG
30,HCLTECH,1943.91,-3.66,36.0,H-LC,9.92,220459.0,-21465.0,84744.0,-1.38,-8.87,38.44,26.15,8.0,-0.25,1.61,4.18,X40,ATH,IT


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SIEMENS,4671.50,-2.83,43.0,H-LC,2.61,156250.0,-29845.0,77328.0,-0.99,-16.04,49.49,25.51,15.0,-0.39,1.14,15.14,AR,ATH,ELECTRICAL
42,ITC,452.00,-39.19,38.0,H-LC,2.20,195771.0,-4367.0,25254.0,0.06,-2.18,12.90,10.44,4.0,-0.17,1.43,3.26,X40,NTT,FMCG
19,CIPLA,1795.00,-23.89,34.0,H-LC,4.51,203418.0,-1082.0,42494.0,-1.65,-0.53,20.89,20.25,10.0,-0.03,1.48,8.16,X40N,BTT,PHARMA
5,ASIANPAINT,3465.66,-17.51,30.0,H-LC,7.05,208146.0,-43622.0,96830.0,-1.61,-17.33,46.52,21.13,27.0,-0.45,1.52,11.96,X40,ATH,PAINTS
57,RELIANCE,1533.00,-14.79,40.0,H-LC,4.06,213424.0,3058.0,25718.0,-0.31,1.45,12.05,13.68,37.0,0.12,1.56,18.22,XY25,NTT,REFINERIES


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4476.75,-30.93,30.0,H-LC,12.07,274997.0,-70959.0,145831.0,-1.08,-20.51,53.03,21.64,1.0,-0.49,2.01,0.00,X40,ATH,IT
40,INFY,2275.00,-20.32,41.0,H-LC,8.12,310788.0,-2100.0,173793.0,-1.73,-0.67,55.92,54.87,3.0,-0.01,2.27,5.93,X40,BTT,IT
42,ITC,452.00,-39.19,38.0,H-LC,2.20,195771.0,-4367.0,25254.0,0.06,-2.18,12.90,10.44,4.0,-0.17,1.43,3.26,X40,NTT,FMCG
83,VBL,671.64,-20.19,32.0,H-LC,8.17,287387.0,-28455.0,141107.0,-0.31,-9.01,49.10,35.67,5.0,-0.20,2.10,3.60,X40N,ATH,FMCG
1,ABB,7934.00,-40.84,42.0,H-LC,7.98,247320.0,-14299.0,133503.0,-0.93,-5.47,53.98,45.57,7.0,-0.11,1.80,5.24,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,COFFEEDAY,80.00,-47.69,42.0,L-SC,41.33,78720.0,-34829.0,74878.0,-3.23,-30.67,95.12,35.27,268.0,-0.47,0.57,92.67,XR,NTT,HOTELS
49,MASFIN,398.61,-20.05,37.0,H-SC,12.45,90600.0,-7380.0,28983.0,-2.20,-7.53,31.99,22.05,152.0,-0.25,0.66,31.69,XR,ATH,FINANCE
50,NATIONALUM,244.55,-45.32,52.0,H-MC,10.98,108364.0,7860.0,21250.0,-0.56,7.82,19.61,28.96,79.0,0.37,0.79,44.44,MH,ATH,METALS
78,UJJIVANSFB,60.00,89.03,46.0,H-SC,10.12,119718.0,-22761.0,42284.0,-1.40,-15.97,35.32,13.70,163.0,-0.54,0.87,41.30,OX40N,NTT,BANKS
79,UNIONBANK,163.00,-14.73,54.0,M-LC,6.84,155144.0,14304.0,30191.0,-1.67,10.16,19.46,31.59,66.0,0.47,1.13,39.33,XY24,NTT,BANKS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SHALBY,327.00,1156.78,62.0,M-SC,12.62,170845.0,-10298.0,55764.0,-1.78,-5.69,32.64,25.10,235.0,-0.18,1.25,36.92,XY24,NTT,HEALTHCARE
71,TANLA,1963.11,-18.28,60.0,H-SC,16.90,219448.0,-51313.0,381269.0,-0.49,-18.95,173.74,121.86,133.0,-0.13,1.60,71.67,AR,ATH,IT
81,VAIBHAVGBL,521.00,54.82,52.0,H-SC,5.61,134226.0,-48549.0,159622.0,-1.26,-26.56,118.92,60.77,125.0,-0.30,0.98,20.22,XR,NTT,JEWELLERY
61,SAMMAANCAP,326.00,-170.45,54.0,M-SC,5.84,81366.0,-20844.0,114238.0,-0.59,-20.39,140.40,91.37,208.0,-0.18,0.59,32.08,XY25,NTT,FINANCE
60,SAIL,228.00,47.55,63.0,M-MC,10.34,232556.0,7594.0,160068.0,-1.08,3.38,68.83,74.53,192.0,0.05,1.70,36.72,XY24,BTT,STEEL


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.27
1,25,43.79
2,50,74.95


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.81
LC    32.45
MC    23.75
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.35
X40      14.78
X40N     11.39
XR       11.13
XY25     10.15
AR        8.33
OX40N     7.89
X200      1.68
MH        1.63
X5K       1.46
SR        1.22
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    29.32
H-LC    25.94
H-MC    20.76
M-SC    13.03
M-LC     5.45
M-MC     2.69
L-SC     1.46
L-LC     1.06
L-MC     0.30
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.07,-6.43,41.56
IT,12.89,-19.13,84.09
FINANCE,10.03,-19.15,71.13
MISC,7.00,-19.68,86.21
BANKS,6.13,-18.63,81.04
PAINTS,5.67,-18.71,36.41
ELECTRICAL,5.47,-9.68,48.16
INSURANCE,3.78,-6.59,46.44
AC,3.59,1.72,14.73


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3219082.0,22
XR,1383762.0,14
AR,1190107.0,9
X40,824150.0,10
X40N,641239.0,9
OX40N,565094.0,10
XY25,447999.0,7
SR,254556.0,2
X5K,117057.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3771477.0,29
M-SC,1458885.0,17
H-LC,1265367.0,15
H-MC,1259608.0,15
M-LC,402051.0,4
M-MC,310122.0,2
L-SC,260859.0,3
L-MC,60875.0,1
L-LC,39467.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1253837.0      6
M-SC       XY24       835401.0      7
H-SC       AR         809011.0      5
           XR         802963.0      7
H-MC       XY24       570963.0      4
H-LC       X40        526452.0      5
H-SC       X40N       316094.0      4
M-MC       XY24       310122.0      2
H-SC       OX40N      266236.0      4
           SR         254556.0      2
H-LC       X40N       241108.0      3
H-MC       X40        221664.0      4
H-LC       AR         210831.0      2
H-MC       XY25       179324.0      2
L-SC       XR         172645.0      2
M-LC       XY24       172388.0      2
M-SC       AR         170265.0      2
           OX40N      138721.0      4
           XR         124226.0      2
M-LC       X5K        117057.0      1
H-LC       XY25       114970.0      3
M-SC       XY25       114238.0      1
M-LC       XR         112606.0      1
H-MC       XR         110447.0      1
H-LC       X200        95635.0      1
L-SC       OX40N       88214.0      1
H-MC       X40N        84037.0      2
H-LC       XY24        76371.0      1
M-SC       X40         76034.0      1
H-MC       OX40N       71923.0      1
H-SC       MH          68780.0      1
L-MC       XR          60875.0      1
L-LC       XY25        39467.0      1
H-MC       MH          21250.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 54.0 seconds
